In [1]:
import cirq
import qiskit as q
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit.circuit.library import PauliEvolutionGate, QFT
from qiskit.synthesis import SuzukiTrotter
from openfermion import QubitOperator


import numpy as np
import scipy, math, csv, os,sys
import matplotlib.pyplot as plt

In [2]:
def convert_op_string_to_usefulForm(op, n):
    '''
    Take a qubit operator operator, e.g. '.5 [Z0 Z1] + 4 [Z0 Z2]' and 
    construct it into a SparsePauliOp form to be used in time evolution
    
    args:
    op: The operator to be converted
    n: Total number of qubits
    
    Yield:
    Op_to_implement: The operator to implement in the circuit (can think of as ~'Hamiltonian')
    '''
    Pauli_list=[]
    
    for i in op:
        temp_str=''
        for j in range(n):
            temp_str+='I'
            
        term=str(i).split(" ")
        coeff = complex(term[0])
        
        term.pop(0)
        for op_term in term:
            if op_term !='[]':
                t_str=op_term.replace('[','')
                t_str=t_str.replace(']','')
                # t_list=t_str.split(" ")
                
                Pauli_op = t_str[0]
                qub_ind = int(t_str[1])
                temp_str = temp_str[::-1] # reverse the string so that python indexing matches qubit labelling
                
                if qub_ind==len(temp_str)-1:
                    temp_str = temp_str[:qub_ind]+Pauli_op
                    # print(qub_ind)
                elif qub_ind<len(temp_str)-1:
                    temp_str = temp_str[:qub_ind]+Pauli_op+temp_str[qub_ind+1:]
                temp_str = temp_str[::-1] # reverse back
        
        
        Pauli_list.append((temp_str,coeff))
            
        
    Op_to_implement=SparsePauliOp.from_list(Pauli_list)
    return Op_to_implement, Pauli_list
        
    

In [3]:
def orderOfMagnitude(number):
    return math.floor(math.log(number, 10))

In [4]:
def one_norm(Pauli_list):
    '''
    Calculate the one-norm of a k-local Hamiltonian
    
    Args:
    ham: Hamiltonian given as a Pauli list
    
    '''
    one_norm = 0
    
    for index in range(len(Pauli_list)):
        sublist = Pauli_list[index]
        
        # Check to see if the k-local operator is non-trivial
        non_triv=0
        for op in sublist[0]:
            if op!= 'I':
                non_triv+=1
        
        if non_triv>0:
            summand = SparsePauliOp.from_list([sublist]) # The individual summand from the qubit operator expression
            summand_as_matrix = summand.to_matrix() 
            # print(summand_as_matrix)
            sub_norm = np.linalg.norm(summand_as_matrix)
            one_norm += sub_norm
       
    
    return one_norm

In [5]:
def q2_upper_bound(p=2,ind=0):
    '''
    Calculate upper bound Trotter number for q2_0 (Hermite and SB)
    
    Arg:
    p: order of the Suzuki-Trotter approximation
    '''
    
    # Calculate 1-norms and induced 1-norms
    
    if ind==0:
        q2_Hermite = QubitOperator("(5+0j) [] + (4+0j) [Z0 Z1]")
        q2_SB = QubitOperator("2.0 [] + -0.5 [Z0] + -0.25881904510252074 [Z0 X1] + 0.9659258262890682 [X1] + -0.9999999999999998 [Z1]")
    
        Hermite_P_list = convert_op_string_to_usefulForm(q2_Hermite, 2)[1]
        Hermite_norm = one_norm(Hermite_P_list)
        print("One-norm for Hermite encoding: ", Hermite_norm)
    
        SB_P_list = convert_op_string_to_usefulForm(q2_SB, 2)[1]
        SB_norm = one_norm(SB_P_list)
        print("One-norm for SB encoding: ", SB_norm)
    
        '''
        The induced 1-norms for q2 Hermite and SB turn out to be the same as the regular one-norm
        '''
        Hermite_ind_norm = Hermite_norm
        SB_ind_norm = SB_norm
    
    elif ind==1:
        q2_Hermite = QubitOperator("(21+0j) [] + (4+0j) [Z0 Z1] + (8+0j) [Z0 Z2] + (16+0j) [Z1 Z2]")
        q2_Gray = QubitOperator("4.0 [] + 1.9777086973269737 [X0 X1] + -1.0117828710379055 [X0 X1 Z2] + -0.9920296962671666 [X0 Z1 X2] + 0.9920296962671666 [X0 X2] + (0.003970132131735471+0j) [Y0 Y1] + (0.25484891297078527+0j) [Y0 Y1 Z2] + (0.1260042924827281+0j) [Y0 Z1 Y2] + (-0.1260042924827281+0j) [Y0 Y2] + -0.5000000000000001 [Z0 Z1 Z2] + -1.0 [Z1 Z2] + -1.9999999999999996 [Z2]")
        
        Hermite_P_list = convert_op_string_to_usefulForm(q2_Hermite, 3)[1]
        Hermite_norm = one_norm(Hermite_P_list)
        print("One-norm for Hermite encoding: ", Hermite_norm)
    
        Gray_P_list = convert_op_string_to_usefulForm(q2_Gray, 3)[1]
        Gray_norm = one_norm(Gray_P_list)
        print("One-norm for Gray encoding: ", Gray_norm)
        
        ## Induced 1-norms
        
        
        
    # Calculate order of Trotter number
    # Assume t=1 for now
    t=1
    err = 1e-6
    Hermite_r = (Hermite_norm * pow(Hermite_ind_norm,(1/p))*pow(t,1+1/p))/pow(err,1/p)
    SB_r = (SB_norm * pow(SB_ind_norm,(1/p))*pow(t,1+1/p))/pow(err,1/p)
    
    print("Hermite encoding: r = O(", orderOfMagnitude(Hermite_r),")")
    print("SB encoding: r = O(", orderOfMagnitude(SB_r),")")
    
    return 

In [6]:
q2_upper_bound()
print()
# q2_upper_bound(ind=1)



One-norm for Hermite encoding:  8.0
One-norm for SB encoding:  5.449489742783177
Hermite encoding: r = O( 4 )
SB encoding: r = O( 4 )



In [7]:
def q3_upper_bound(p=2):
    '''
    Calculate upper bound Trotter number for q3_0 (Hermite and SB)
    '''
    
    q3_Hermite = QubitOperator("(41+0j) [] + (40+0j) [Z0 Z1]")
    q3_SB = QubitOperator("3.286006046520985 [X0] + 1.933012701892219 [X0 X1] + -2.225345874741164 [X0 Z1] + (1.0669872981077806+0j) [Y0 Y1]")
    
    Hermite_P_list = convert_op_string_to_usefulForm(q3_Hermite, 2)[1]
    Hermite_norm = one_norm(Hermite_P_list)
    print("One-norm for Hermite encoding: ", Hermite_norm)
    
    SB_P_list = convert_op_string_to_usefulForm(q3_SB, 2)[1]
    SB_norm = one_norm(SB_P_list)
    print("One-norm for SB encoding: ", SB_norm)
    
    '''
    The induced 1-norms for q3
    '''
    Hermite_ind_norm = Hermite_norm
    SB_ind_norm = SB_norm
    
    # Calculate order of Trotter number
    # Assume t=1 for now
    t=1
    err = 1e-6
    Hermite_r = (Hermite_norm * pow(Hermite_ind_norm,(1/p))*pow(t,1+1/p))/pow(err,1/p)
    SB_r = (SB_norm * pow(SB_ind_norm,(1/p))*pow(t,1+1/p))/pow(err,1/p)
    
    print("Hermite encoding: r = O(", orderOfMagnitude(Hermite_r),")")
    print("SB encoding: r = O(", orderOfMagnitude(SB_r),")")
    
    return

In [8]:
q3_upper_bound()

One-norm for Hermite encoding:  80.0
One-norm for SB encoding:  17.022703842524297
Hermite encoding: r = O( 5 )
SB encoding: r = O( 4 )


In [9]:
def calc_max_locality(ham,n):
    '''
    Determine the maximum locality of a given Hamiltonian operator
    
    Args:
    ham: Hamiltonian
    n: number of qubits
    
    Yield:
    k: The max locality
    '''
    
    P_list = convert_op_string_to_usefulForm(ham, n)[1]
    
    k=0
    for index in range(len(P_list)):
        sublist = P_list[index]
        
        # Check to see if the k-local operator is non-trivial
        non_triv=0
        for op in sublist[0]:
            if op!= 'I':
                non_triv+=1
        k=max(k,non_triv)
        
    
    
    return k

In [10]:
# q2_Hermite = QubitOperator("(5+0j) [] + (4+0j) [Z0 Z1]")
# q2_SB = QubitOperator("2.0 [] + -0.5 [Z0] + -0.25881904510252074 [Z0 X1] + 0.9659258262890682 [X1] + -0.9999999999999998 [Z1]")
# t1 = calc_max_locality(q2_Hermite,2)
# t2 = calc_max_locality(q2_SB,2)
# print(t1,t2)

In [11]:
# q3_Hermite = QubitOperator("(41+0j) [] + (40+0j) [Z0 Z1]")
# q3_SB = QubitOperator("3.286006046520985 [X0] + 1.933012701892219 [X0 X1] + -2.225345874741164 [X0 Z1] + (1.0669872981077806+0j) [Y0 Y1]")
# t3 = calc_max_locality(q3_Hermite,2)
# t4 = calc_max_locality(q3_SB,2)
# print(t3,t4)

In [15]:
def update_all_single_ops_circuits():
    '''
    Add maximum locality to single_op_df_updated_v3.csv
    '''
    
    Circuit_list=[]
    
    # Specify the relative path to the CSV file
    # relative_path = "single_op_df_updated_v3.csv"
    relative_path = "single_op_df_updated_v5.csv"
    # relative_path = "csv_files/128_single_op_df.csv"

    # Get the absolute path of the CSV file
    csv_path = os.path.abspath(relative_path)
    
    # Specify the relative path to the new CSV file with the added column
    # relative_output_path = "single_op_df_updated_v4.csv"
    relative_output_path = "single_op_df_updated_v6.csv"
    # relative_output_path = "128_single_op_df_updated.csv"

    # Get the absolute path of the new CSV file
    output_csv_path = os.path.abspath(relative_output_path)
    
    with open(csv_path, 'r') as infile, open(output_csv_path, 'w', newline='') as outfile:
        csv_reader = csv.reader(infile)
        csv_writer = csv.writer(outfile)
        
        # Read the header row from the original CSV file
        header = next(csv_reader)
    
        # Add the name of the new column to the header
        header.append("Maximum Locality") 
        
        # Write the updated header to the new CSV file
        csv_writer.writerow(header)
    
        # Iterate through each row in the CSV file
        for row in csv_reader:
            # Access values in each row by index or column name
            op_name = row[0]  # Access the first column (Column1)
            d_val_str = row[1]  # Access the second column (Column2)
            d_val = list(map(int, d_val_str[1:-1].split(',')))
            enc_name = row[2]  # Access the third column (Column3)
            qub_operator = QubitOperator(row[3])
            Trotter_info = row[4]
            
            num_qubits = int(math.log(d_val[0],2))
            
            k = calc_max_locality(qub_operator,num_qubits)
            
            # Modify "new_data" with the information you want to add to the new column
            row.append(k)
        
            # Write the updated row to the new CSV file
            csv_writer.writerow(row)   
                
            
    return 
            
            
            
            
            
            
    

In [16]:
update_all_single_ops_circuits()